# use md5sum to determine sample swapping

2021.8.18, fzz

## Workflow:
1. get all md5sums for all fastqs in Rusty
2. get all md5sums within snakemake pipeline (should be a subset of 1)
3. use 1, pair files with the same md5
4. use master v7 -> v9 to resolve the paired order
6. map back to 2 to determine problematic sids
5. report

In [1]:
def get_sid(f):
    sample_name_ele = os.path.basename(f).rstrip(".fastq.gz").split('_')
    if sample_name_ele[-2].startswith("P"):
        sample_id = "_".join(sample_name_ele[:-2])
        sample_id_w_plate = "_".join(sample_name_ele[:-1])
    else:
        sample_id = "_".join(sample_name_ele[:-1])
        sample_id_w_plate = sample_id
    return sample_id, sample_id_w_plate

In [2]:
import pandas as pd
import os
from collections import defaultdict, Counter

# read in md5, everything downloaded to Rusty as of Aug 18, 2021
par_dir = '/mnt/ceph/users/zzhang/My-RNASeq-pipeline/NAVY_CHARM_20200816/md5sums'
md5_files = [x for x in os.listdir(par_dir) if x.endswith('.md5')]
md5_to_file = defaultdict(set)
md5_to_sid = defaultdict(set)
file_to_md5 = defaultdict(set)
for f in md5_files:
    with open(os.path.join(par_dir, f), 'r') as fh:
        for line in fh:
            ele = line.strip().split()
            sid, sid_p = get_sid(ele[1])
            md5_to_sid[ele[0]].add(sid_p)
            md5_to_file[ele[0]].add(ele[1])
            file_to_md5[sid_p].add(ele[0])


# Read in Metadata v7 and v9

In [3]:
# read in meta_7 v7
meta_7 = pd.read_table('/mnt/ceph/users/zzhang/jemm/data-V7/charm_master.clean.csv', index_col=0, sep=",", low_memory=False)
print(meta_7.shape) 
meta_7 = meta_7[meta_7.plateNum.notna()]
print(meta_7.shape)
sero = pd.read_table('/mnt/ceph/users/zzhang/jemm/data-V7.p1/merged_seropos.20210814.txt', index_col=0)
meta_7 = meta_7[meta_7.pid.isin(sero.query('fz_label=="keep"').pid)]
print(meta_7.shape)
meta_7 = meta_7[meta_7.final.isin(['Control', 'First', 'Mid', 'Post'])]
print(meta_7.shape)

(1176, 11)
(1176, 11)
(825, 11)
(763, 11)


In [4]:
meta_7[['pid','Sex','final']].drop_duplicates().groupby(['final','Sex']).size()

final    Sex
Control  F       30
         M      147
First    F       28
         M       34
Mid      F       23
         M      103
Post     F       18
         M      109
dtype: int64

In [5]:
# read in meta_9 v9.1
meta_9 = pd.read_table('/mnt/ceph/users/zzhang/jemm/data-V9/charm_master.csv', index_col=0, sep="\t", low_memory=False)
print(meta_9.shape)  # 19604, 69
meta_9 = meta_9[meta_9.RNAseq_plate.notna()] # 1926, 69
print(meta_9.shape)
sero = pd.read_table('/mnt/ceph/users/zzhang/jemm/data-V7.p1/merged_seropos.20210814.txt', index_col=0)
meta_9 = meta_9[meta_9.pid.isin(sero.query('fz_label=="keep"').pid)] # 1470, 69
print(meta_9.shape)
meta_9 = meta_9[meta_9.final.isin(['Control', 'First', 'Mid', 'Post'])] # 1332, 69
print(meta_9.shape)

(19604, 69)
(1926, 69)
(1470, 69)
(1332, 69)


In [6]:
meta_9[['pid','Sex','final']].drop_duplicates().groupby(['final','Sex']).size()

final    Sex
Control  F       38
         M      232
First    F       32
         M       58
Mid      F       26
         M      177
Post     F       25
         M      176
dtype: int64

In [7]:
redup = []
sid_SampleMismatch = {}
sid_TimeMismatch = {}
for md5 in md5_to_sid:
    if len(md5_to_sid[md5]) > 1:
        redup.append(md5)
        sids = [x for x in md5_to_sid[md5]]
        sams = set([x.split('-')[0] for x in sids])
        if len(sams) > 1:
            sid_SampleMismatch[md5] = sids
        elif len(set(sids)) > 1:
            sid_TimeMismatch[md5] = sids

In [8]:
len(redup)

52

In [9]:
md5_to_file[redup[0]]

{'/mnt/ceph/users/zzhang/COVID/Navy_Charm_v7.20210120/delta_files/delta_fastq/20_0262-T42_P6_R1.fastq.gz',
 '/mnt/ceph/users/zzhang/COVID/gey01.u.hpc.mssm.edu/Covid19/RNA-seq/charm_fastq/P1_P8/20_0262-T40_R1.fastq.gz'}

In [10]:
sid_SampleMismatch

{'d29ac0b9c1a6102d0fb71914ebb4fcf4': ['20_0575-T49_P9', '20_0585-T49'],
 'a594d22ac91d21dc798155a215299ecb': ['20_0575-T49_P9', '20_0585-T49'],
 'c7044d7a2985c53518805c8e7b70e4c8': ['20_0625-T56_P10', '20_0627-T56'],
 'b141c043e60590cea9c016488d436c03': ['20_0625-T56_P10', '20_0627-T56'],
 '5bb9b8fed491be2d72455572b4947a2f': ['20_0195-T28', '20_0208-T28_P5'],
 '51e7647114484231759772168740db11': ['20_0195-T28', '20_0208-T28_P5']}

In [11]:
def resolve_file_order(SampleMismatch, meta_7, meta_9):
    #SampleMismatch = sid_SampleMismatch['c7044d7a2985c53518805c8e7b70e4c8']
    sids = ['20_'+x.split('_')[1] if x.split('_')[-1].startswith('P') else x for x in SampleMismatch]
    print([(x,y) for x, y in zip(sids, SampleMismatch)])
    sids_7 = {y:'v7' if x in meta_7.index else '~v7' for x, y in zip(sids, SampleMismatch)}
    sids_9 = {y:'v9' if x in meta_9.index else '~v9' for x, y in zip(sids, SampleMismatch)}

    print(sids_7, sids_9)
    return

In [12]:
resolve_file_order(sid_SampleMismatch['5bb9b8fed491be2d72455572b4947a2f'], meta_7, meta_9)

[('20_0195-T28', '20_0195-T28'), ('20_0208-T28', '20_0208-T28_P5')]
{'20_0195-T28': '~v7', '20_0208-T28_P5': '~v7'} {'20_0195-T28': '~v9', '20_0208-T28_P5': '~v9'}


In [13]:
sid_TimeMismatch

{'a45fa254dd09aed606ef849ce0955669': ['20_0262-T42_P6', '20_0262-T40'],
 '128c31f3949bec1ea7b3b4810d169ad6': ['20_0262-T42_P6', '20_0262-T40'],
 '8fdbe55003cb4a3363a73402924ad17d': ['20_0267-T39_P13', '20_0267-T40_P13'],
 '5861ca77986dc2189968ff2f5cbb59a8': ['20_0267-T39_P13', '20_0267-T40_P13'],
 'fec82805ca07ca1331f1c0c8cf9390be': ['20_0267-T40_P6',
  '20_0267-T04',
  '20_0267-T39_P6'],
 'dff709fee605b61107f78e83f5449aa3': ['20_0267-T40_P6',
  '20_0267-T04',
  '20_0267-T39_P6'],
 '3616a273fb462d4715177ee1afe0a1e3': ['20_0267-T42_P6', '20_0267-T07'],
 '6fe0c4ee65ab7483840208587a4ea239': ['20_0267-T42_P6', '20_0267-T07'],
 '0d3c9e9dfe6b802b62de4ef25bc127ad': ['20_0267-T46_P7', '20_0267-T10'],
 '644f7d9bd941025fdeecffefb558b64d': ['20_0267-T46_P7', '20_0267-T10'],
 'd7d9229c2c9c1e29d36afd2793c6d8a4': ['20_0811-T13', '20_0811-T14_P5'],
 'cc667a7438b3f6890906834bb9d71810': ['20_0811-T13', '20_0811-T14_P5'],
 'e2247798391ed663188166f26b79c597': ['20_0813-T14_P5', '20_0813-T13'],
 '75a8aee5